# Setting up the environment

You can choose one of the following ways to create your virtual environment. Virtual environments are useful so you do not litter your system python installation. All commands executed will use relative paths from your current directory.

## Anaconda (option 1)

You can search the documentation of the following commands here:  
https://docs.conda.io/en/latest/

(Update conda to the latest version: `conda update conda`)

(Maybe it is required to run `conda init <SHELL_NAME>` where the shell name could be e.g: bash, powershell, etc.)

Creating a new Anaconda environment (places the new env in the default directory):  
`conda create --name python_testing`

Activate the new environment:  
`conda activate python_testing`

(Note: if you want to leave this virtualenv just run `conda deactivate`)

Installing packages:  
`conda install jupyter pytest selenium`

## Python virtualenv (option 2)

I assume you have python3 installed with pip. If not, than install these (I think the default python installation  contains the pip package, but if not, then install it separately):  
https://www.python.org/  
https://pip.pypa.io/en/stable/installing/

You can read more about virtualenvs here:  
https://virtualenv.pypa.io/en/latest/

Download virtualenv package:  
`python3 -m pip install virtualenv`

Create virtualenv for this session (this creates the virtualenv in a directory called `venv` under the current directory):  
`virtualenv venv/python_testing`

Activating the virtualenv under Linux or OSX:  
`source venv/python_testing/activate`

Activating the virtualenv under Windows:
`.\venv\python_testing\Scripts\activate`

Running the above command causes your prompt to change: it should have the name of your virtualenv before your usual prompt (`(python_testing)`). Note: if you want to leave this virtual env, just run `deactivate`, and your prompt will be changed back to normal.

Installing the necessary packages (in an activated virtualenv):  
`python3 -m pip install jupyter pytest selenium`


## Common: downloading geckodriver

In this session we will use geckodriver, the selenium webdriver implementation for Firefox. Chrome users can find chromedriver, and other browser users can find their respective drivers as well.

Create a directory for this session (if you haven't already), and create a subdirectory called `bin` under it.  
NOTE: the virtualenv directory and the working directory should not be the same.

Download the latest version of geckodriver (to drive Firefox browser):  
https://github.com/mozilla/geckodriver/releases/

Extract and put the binary in the previously created `bin` directory.

If you want to run this notebook on your own computer, download the file, put it into your working directory, activate the virtualenv, and run the command:  
`jupyter-notebook`

# Let the testing begin

## Do the imports

If the imports make an Exception, then the setup of the environment was not successful. You should try to fix it before continuing.

In [4]:
import pytest
from time import sleep
from selenium import webdriver

## Create simple testcase as a function

This simple testcase could be used by the wikipedia admins, that the article about the Chernobyl disaster is not tampered with. Of course this test is just for presentation purposes.

The python selenium documentation can be found here:  
https://selenium-python.readthedocs.io/

The 4th section "Locating elements" explains the different element finding methods.

Locating these descriptors can be easily done by using the Developer Toolbar of the browser of your choice, usually it can be activated with the F12 button.

The most important part of the following function is the `assert` statement. If the expression written after it is False, then it raises an AssertionError. This will make the test fail.

In [5]:
def test_chernobyl_fact(fact_in_first_paragraph):
    # The driver represents the object, which can be used to interact with the browser.
    # Note to Windows users: don't forget to add .exe to the executable path.
    driver = webdriver.Firefox(executable_path="bin/geckodriver")
    # The get method can be used to load a webpage in the browser
    driver.get("https://en.wikipedia.org/wiki/Main_Page")
    # You can place sleeps in the test to see how the driver interacts with the browser.
    # The next line will wait 5 seconds before continuing the execution.
    sleep(5)
    # The driver searches for the HTML element with id "searchInput". If it does not find it it drops an exception.
    search_field = driver.find_element_by_id("searchInput")
    # We enter the search string.
    search_field.send_keys("chernobyl disaster")
    # We locate the searchButton.
    search_button = driver.find_element_by_id("searchButton")
    # We click on the search_button.
    search_button.click()
    # The bodyContent will contain the necessary information
    body_content = driver.find_element_by_id("bodyContent")
    # This next find will return a list of all the "p" tags. If there isn't any "p" tag, it returns an empty list.
    paragraphs = body_content.find_elements_by_tag_name("p")
    # This is where actually the testing happens. If there is an error before this, then it is a badly written test
    # and the test is faulty, if this fails, then the tested functionality is at fault.
    # We check for the fact to be a part of the second paragraph's text property.
    assert fact_in_first_paragraph in paragraphs[2].text
    # The return 0 is used to see the return value, when the function is called.
    return 0

## Check the date

In [6]:
test_chernobyl_fact("26 April 1986")

0

Notice the still open firefox window? It is because we forgot to close it with the driver. I deleted the `sleep` from the code.

In [7]:
def test_chernobyl_fact(fact_in_first_paragraph):
    driver = webdriver.Firefox(executable_path="bin/geckodriver")
    driver.get("https://en.wikipedia.org/wiki/Main_Page")
    search_field = driver.find_element_by_id("searchInput")
    search_field.send_keys("chernobyl disaster")
    search_button = driver.find_element_by_id("searchButton")
    search_button.click()
    body_content = driver.find_element_by_id("bodyContent")
    paragraphs = body_content.find_elements_by_tag_name("p")
    assert fact_in_first_paragraph in paragraphs[2].text
    driver.close()  # This will close the browser.
    return 0
test_chernobyl_fact("26 April 1986")

0

## Check some incorrect fact

In [8]:
test_chernobyl_fact("US spies")

AssertionError: 

The above code should raise an AssertionError, this is expected. Because the code never reached the `driver.close()` line, the browser remains open. This will be resolved later.

## Let's write a test that finds the date of the Fukushima nuclead disaster from the Chernobyl nuclear disaster page

This part should be written by you, and later compared with the produced solution.

In [9]:
def test_fukushima():
    driver = webdriver.Firefox(executable_path="bin/geckodriver")
    driver.get("https://en.wikipedia.org/wiki/Chernobyl_disaster")
    # The find_elements_by_xpath returns a list, and we get the first element out of this list with the 0 index.
    link = driver.find_elements_by_xpath("//a[@href='/wiki/Fukushima_Daiichi_nuclear_disaster']")[0]
    link.click()
    # Here another method is used to find the required element: css_selector.
    p = driver.find_element_by_css_selector(".mw-parser-output > p:nth-child(8)")
    assert "11 March 2011" in p.text
    driver.close()
    return 0

In [10]:
test_fukushima()

0

## Move to a separate file

Normally testing isn't done in a Jupyter notebook file, but in a separate .py file. So open now the `web.py` file found next to this notebook file. The two functions are simplified, the unnecessary parts are cut out. The code is explained in the comments.

To run the tests with pytest, open a terminal, and activate the virtualenv and enter the directory. The tests can be run with the following command:  
`python3 -m pytest web.py`

The output should be similar to this:
```
========================================== test session starts ==========================================
platform linux -- Python 3.7.4, pytest-5.2.0, py-1.8.0, pluggy-0.13.0
rootdir: /home/zsoltendreffy/dev/python_testing
collected 3 items                                                                                                                     

web.py .x.                                                                                                                      [100%]

===================================== 2 passed, 1 xfailed in 9.02s ======================================```

There is also another file, `web_unittest.py`. It contains almost the same tests, but written with the python unittest framework.

To run this file you can use the same terminal, you used for the pytest version. Run with the following command:  
`python3 -m unittest web_unittest.py`

The output:  
```
s.x.
----------------------------------------------------------------------
Ran 4 tests in 25.120s

OK (skipped=1, expected failures=1)
```